In [15]:
from theano.sandbox import cuda
cuda.use('gpu')

In [3]:
import keras
from keras.layers.normalization import BatchNormalization
import keras.layers
from keras.layers import Dropout

Using Theano backend.


In [4]:

from utils import *
%matplotlib inline

In [5]:
import os,sys
current_dir = os.getcwd()
Data_dir = current_dir+'/data/statefarm/'

In [6]:
#path = Data_dir+'/sample/' #sample path
path = Data_dir

In [7]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]
def load_carray(fname): return bcolz.open(fname)

In [8]:
batch_size=64

In [9]:
#generate an augmented train batch
imgGen = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05,shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

augmentTrainBatch = imgGen.flow_from_directory(directory=path+'train', batch_size=64, target_size=(224,224), shuffle=False, class_mode='categorical')
              

Found 20244 images belonging to 10 classes.


In [10]:
#generate a new validation batch
batches =augmentTrainBatch


In [11]:
#display images
img,lbl = val_batches.next()
plots(img[:4], titles=hotone(lbl[:4]))

NameError: name 'val_batches' is not defined

In [ ]:
val_batches.reset()

In [9]:
val_batches = get_batches(path+'valid', batch_size=batch_size, shuffle=False,  target_size=(224,224))

Found 2180 images belonging to 10 classes.


In [63]:
train_label_arr=onehot(batches.classes)
train_file_names_arr=batches.filenames


In [11]:
valid_label_arr=onehot(val_batches.classes)
valid_file_names_arr=val_batches.filenames

In [50]:
test_batch =  get_batches(path+'test', batch_size=batch_size, shuffle=False,  target_size=(224,224), class_mode='binary')
test_file_names_arr=test_batch.filenames

Found 79726 images belonging to 1 classes.


In [64]:

save_array("train_label_arr", train_label_arr)
save_array("train_file_names_arr", train_file_names_arr)

In [14]:
save_array("valid_label_arr", valid_label_arr)
save_array("valid_file_names_arr", valid_file_names_arr)

In [15]:
save_array("test_file_names_arr", test_file_names_arr)

In [15]:
train_label_arr = load_array("train_label_arr")
train_file_names_arr = load_array("train_file_names_arr")

In [16]:
valid_label_arr = load_array("valid_label_arr")
valid_file_names_arr = load_array("valid_file_names_arr")

In [65]:
train_label_arr.shape

(20244, 10)

In [18]:
import vgg16

In [19]:
vggModel=Vgg16()

In [ ]:
vggModel.model.summary()

In [21]:
lastConvIndex = -1
for i in range(len(vggModel.model.layers)):
    if (type(vggModel.model.layers[i]) == Convolution2D):
        lastConvIndex=i
        
print ('last conv layer index ' + str(i) )

last conv layer index 37


In [22]:
convLayers= vggModel.model.layers[:lastConvIndex+1]

In [23]:

vggConvLayersModel = Sequential(convLayers)

In [24]:
batch_size=32

In [78]:
NepredictTrainVggConv = vggConvLayersModel.predict_generator(batches, batches.N)

In [28]:

predictValidationVggConv = vggConvLayersModel.predict_generator(val_batches, val_batches.N)

In [16]:
predict_test_on_VggConv = vggConvLayersModel.predict_generator(test_batch, test_batch.N)

In [79]:
save_array("NepredictTrainVggConv", NepredictTrainVggConv)

In [29]:
save_array("predictValidationVggConv", predictValidationVggConv)

In [32]:
predictValidationVggConv = load_carray("predictValidationVggConv")

In [8]:
NepredictTrainVggConv = load_carray("NepredictTrainVggConv")

In [ ]:
predictTrainVggConv[:5]

In [ ]:
predict_test_on_VggConv

In [17]:
save_array("predict_test_on_VggConv", predict_test_on_VggConv)

In [51]:
predict_test_on_VggConv = load_carray("predict_test_on_VggConv")

In [69]:
NewpredictTrainVggConv.shape

(20244, 512, 14, 14)

In [45]:
def genDenseModel(layer):
    baseModel = Sequential()
    baseModel.add(MaxPooling2D(input_shape=layer.output_shape[1:]))
    #baseModel.add(Convolution2D(32, 3, 3, activation='relu'))
    #baseModel.add(BatchNormalization(axis=1))
    #baseModel.add(MaxPooling2D(pool_size=(2,2)))
    #baseModel.add(Convolution2D(64, 3, 3, activation='relu'))
    #baseModel.add(BatchNormalization(axis=1))
    #baseModel.add(MaxPooling2D(pool_size=(2,2)))
    baseModel.add(Flatten())
    baseModel.add(Dropout(0.5))
    baseModel.add(Dense(256,activation='relu'))
    baseModel.add(BatchNormalization())
    baseModel.add(Dropout(0.5))
    baseModel.add(Dense(256,activation='relu'))
    baseModel.add(BatchNormalization())
    baseModel.add(Dropout(0.8))
    baseModel.add(Dense(10,activation='softmax'))
    
    baseModel.compile(Adam(lr=0.0001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return baseModel

In [82]:
denseModel = genDenseModel(vggConvLayersModel.layers[-1])

In [31]:
def get_fc_model():
    model = Sequential([MaxPooling2D(input_shape=convLayers[-1].output_shape[1:]),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.),
        Dense(256, activation='relu'),
        Dropout(0.),
        Dense(10, activation='softmax')])

    model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model
 

In [84]:
denseModel.compile(Adam(lr=0.001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [85]:
denseModel.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=3,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/3
20244/20244 [==============================] - 27s - loss: 2.8553 - acc: 0.2654 - val_loss: 1.1232 - val_acc: 0.6197
Epoch 2/3
20244/20244 [==============================] - 28s - loss: 1.4264 - acc: 0.4691 - val_loss: 1.0929 - val_acc: 0.6041
Epoch 3/3
20244/20244 [==============================] - 28s - loss: 1.1202 - acc: 0.5421 - val_loss: 0.9790 - val_acc: 0.6560


In [86]:
denseModel.compile(Adam(lr=0.0001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModel.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=10,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/10
20244/20244 [==============================] - 28s - loss: 0.9883 - acc: 0.5834 - val_loss: 0.9767 - val_acc: 0.6307
Epoch 2/10
20244/20244 [==============================] - 28s - loss: 0.9461 - acc: 0.5971 - val_loss: 0.9736 - val_acc: 0.6422
Epoch 3/10
20244/20244 [==============================] - 28s - loss: 0.9127 - acc: 0.6112 - val_loss: 0.9693 - val_acc: 0.6321
Epoch 4/10
20244/20244 [==============================] - 28s - loss: 0.8851 - acc: 0.6263 - val_loss: 0.9303 - val_acc: 0.6587
Epoch 5/10
20244/20244 [==============================] - 28s - loss: 0.8553 - acc: 0.6280 - val_loss: 0.9725 - val_acc: 0.6500
Epoch 6/10
20244/20244 [==============================] - 28s - loss: 0.8430 - acc: 0.6373 - val_loss: 0.9673 - val_acc: 0.6491
Epoch 7/10
20244/20244 [==============================] - 28s - loss: 0.8207 - acc: 0.6487 - val_loss: 0.9409 - val_acc: 0.6619
Epoch 8/10
20244/20244 [==============================]

In [87]:
denseModel.compile(Adam(lr=0.0001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModel.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=20,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/20
20244/20244 [==============================] - 28s - loss: 0.7584 - acc: 0.6752 - val_loss: 1.0014 - val_acc: 0.6326
Epoch 2/20
20244/20244 [==============================] - 28s - loss: 0.7506 - acc: 0.6761 - val_loss: 0.9448 - val_acc: 0.6546
Epoch 3/20
20244/20244 [==============================] - 28s - loss: 0.7366 - acc: 0.6815 - val_loss: 1.0109 - val_acc: 0.6394
Epoch 4/20
20244/20244 [==============================] - 28s - loss: 0.7206 - acc: 0.6871 - val_loss: 0.9468 - val_acc: 0.6560
Epoch 5/20
20244/20244 [==============================] - 28s - loss: 0.7032 - acc: 0.6991 - val_loss: 0.9711 - val_acc: 0.6514
Epoch 6/20
20244/20244 [==============================] - 28s - loss: 0.6972 - acc: 0.7044 - val_loss: 0.9789 - val_acc: 0.6541
Epoch 7/20
20244/20244 [==============================] - 28s - loss: 0.6793 - acc: 0.7092 - val_loss: 0.9469 - val_acc: 0.6784
Epoch 8/20
20244/20244 [==============================]

In [88]:
denseModel.compile(Adam(lr=0.0001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModel.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=20,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/20
20244/20244 [==============================] - 28s - loss: 0.5373 - acc: 0.7796 - val_loss: 1.0253 - val_acc: 0.6133
Epoch 2/20
20244/20244 [==============================] - 28s - loss: 0.5254 - acc: 0.7859 - val_loss: 1.0368 - val_acc: 0.6229
Epoch 3/20
20244/20244 [==============================] - 28s - loss: 0.5227 - acc: 0.7898 - val_loss: 1.0777 - val_acc: 0.5945
Epoch 4/20
20244/20244 [==============================] - 28s - loss: 0.5157 - acc: 0.7923 - val_loss: 1.0570 - val_acc: 0.5995
Epoch 5/20
20244/20244 [==============================] - 28s - loss: 0.4978 - acc: 0.7961 - val_loss: 1.1193 - val_acc: 0.5936
Epoch 6/20
20244/20244 [==============================] - 28s - loss: 0.5023 - acc: 0.7950 - val_loss: 1.1177 - val_acc: 0.6037
Epoch 7/20
20244/20244 [==============================] - 28s - loss: 0.4971 - acc: 0.8023 - val_loss: 1.1691 - val_acc: 0.5757
Epoch 8/20
20244/20244 [==============================]

In [90]:
denseModel.compile(Adam(lr=0.0001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModel.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=20,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/20
20244/20244 [==============================] - 29s - loss: 0.3125 - acc: 0.8831 - val_loss: 1.2832 - val_acc: 0.5867
Epoch 2/20
20244/20244 [==============================] - 29s - loss: 0.3060 - acc: 0.8850 - val_loss: 1.2554 - val_acc: 0.5954
Epoch 3/20
20244/20244 [==============================] - 29s - loss: 0.2948 - acc: 0.8930 - val_loss: 1.3374 - val_acc: 0.5734
Epoch 4/20
20244/20244 [==============================] - 29s - loss: 0.2938 - acc: 0.8919 - val_loss: 1.2466 - val_acc: 0.5982
Epoch 5/20
20244/20244 [==============================] - 30s - loss: 0.2925 - acc: 0.8932 - val_loss: 1.2844 - val_acc: 0.5706
Epoch 6/20
20244/20244 [==============================] - 29s - loss: 0.2919 - acc: 0.8917 - val_loss: 1.2277 - val_acc: 0.5894
Epoch 7/20
20244/20244 [==============================] - 30s - loss: 0.2856 - acc: 0.8950 - val_loss: 1.2174 - val_acc: 0.5862
Epoch 8/20
20244/20244 [==============================]

In [91]:
denseModel.compile(Adam(lr=0.0001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModel.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=7,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/7
20244/20244 [==============================] - 29s - loss: 0.2483 - acc: 0.9088 - val_loss: 1.2272 - val_acc: 0.5986
Epoch 2/7
20244/20244 [==============================] - 30s - loss: 0.2438 - acc: 0.9105 - val_loss: 1.1445 - val_acc: 0.6128
Epoch 3/7
20244/20244 [==============================] - 29s - loss: 0.2369 - acc: 0.9129 - val_loss: 1.2130 - val_acc: 0.6009
Epoch 4/7
20244/20244 [==============================] - 29s - loss: 0.2401 - acc: 0.9121 - val_loss: 1.2378 - val_acc: 0.6000
Epoch 5/7
20244/20244 [==============================] - 29s - loss: 0.2413 - acc: 0.9121 - val_loss: 1.1253 - val_acc: 0.6248
Epoch 6/7
20244/20244 [==============================] - 29s - loss: 0.2365 - acc: 0.9122 - val_loss: 1.1304 - val_acc: 0.6275
Epoch 7/7
20244/20244 [==============================] - 29s - loss: 0.2333 - acc: 0.9156 - val_loss: 1.1632 - val_acc: 0.6225


In [93]:
denseModel.compile(Adam(lr=0.0001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModel.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=15,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/15
20244/20244 [==============================] - 30s - loss: 0.2066 - acc: 0.9264 - val_loss: 1.3958 - val_acc: 0.5720
Epoch 2/15
20244/20244 [==============================] - 29s - loss: 0.2041 - acc: 0.9271 - val_loss: 1.3929 - val_acc: 0.5858
Epoch 3/15
20244/20244 [==============================] - 29s - loss: 0.2105 - acc: 0.9262 - val_loss: 1.5091 - val_acc: 0.5761
Epoch 4/15
20244/20244 [==============================] - 29s - loss: 0.2003 - acc: 0.9300 - val_loss: 1.3784 - val_acc: 0.5839
Epoch 5/15
20244/20244 [==============================] - 29s - loss: 0.2105 - acc: 0.9255 - val_loss: 1.4489 - val_acc: 0.5642
Epoch 6/15
20244/20244 [==============================] - 29s - loss: 0.2021 - acc: 0.9277 - val_loss: 1.4371 - val_acc: 0.5697
Epoch 7/15
20244/20244 [==============================] - 28s - loss: 0.1965 - acc: 0.9294 - val_loss: 1.4346 - val_acc: 0.5509
Epoch 8/15
20244/20244 [==============================]

In [97]:
denseModel.compile(Adam(lr=0.01), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModel.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=5,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/5
20244/20244 [==============================] - 28s - loss: 1.5784 - acc: 0.4483 - val_loss: 1.2157 - val_acc: 0.5936
Epoch 2/5
20244/20244 [==============================] - 28s - loss: 1.0813 - acc: 0.5648 - val_loss: 2.7458 - val_acc: 0.4206
Epoch 3/5
20244/20244 [==============================] - 28s - loss: 0.9834 - acc: 0.5927 - val_loss: 1.0835 - val_acc: 0.6138
Epoch 4/5
20244/20244 [==============================] - 28s - loss: 0.9477 - acc: 0.6061 - val_loss: 1.1771 - val_acc: 0.5683
Epoch 5/5
20244/20244 [==============================] - 29s - loss: 0.9016 - acc: 0.6192 - val_loss: 0.9264 - val_acc: 0.6904


In [95]:
denseModel.compile(Adam(lr=0.00001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModel.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=10,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/10
20244/20244 [==============================] - 28s - loss: 0.1725 - acc: 0.9374 - val_loss: 1.3392 - val_acc: 0.5748
Epoch 2/10
20244/20244 [==============================] - 28s - loss: 0.1689 - acc: 0.9408 - val_loss: 1.3495 - val_acc: 0.5789
Epoch 3/10
20244/20244 [==============================] - 29s - loss: 0.1711 - acc: 0.9389 - val_loss: 1.3355 - val_acc: 0.5821
Epoch 4/10
20244/20244 [==============================] - 29s - loss: 0.1581 - acc: 0.9435 - val_loss: 1.3218 - val_acc: 0.5844
Epoch 5/10
20244/20244 [==============================] - 28s - loss: 0.1563 - acc: 0.9428 - val_loss: 1.3544 - val_acc: 0.5798
Epoch 6/10
20244/20244 [==============================] - 29s - loss: 0.1581 - acc: 0.9437 - val_loss: 1.3300 - val_acc: 0.5812
Epoch 7/10
20244/20244 [==============================] - 29s - loss: 0.1566 - acc: 0.9442 - val_loss: 1.3245 - val_acc: 0.5803
Epoch 8/10
20244/20244 [==============================]

In [98]:
denseModel.compile(Adam(lr=0.001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModel.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=5,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/5
20244/20244 [==============================] - 28s - loss: 0.7995 - acc: 0.6629 - val_loss: 0.9198 - val_acc: 0.6839
Epoch 2/5
20244/20244 [==============================] - 28s - loss: 0.7354 - acc: 0.6841 - val_loss: 0.9877 - val_acc: 0.6784
Epoch 3/5
20244/20244 [==============================] - 28s - loss: 0.7040 - acc: 0.6994 - val_loss: 0.9931 - val_acc: 0.6642
Epoch 4/5
20244/20244 [==============================] - 28s - loss: 0.6853 - acc: 0.7043 - val_loss: 0.9228 - val_acc: 0.6917
Epoch 5/5
20244/20244 [==============================] - 28s - loss: 0.6617 - acc: 0.7129 - val_loss: 0.9508 - val_acc: 0.6817


In [99]:
denseModel.compile(Adam(lr=0.0001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModel.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=25,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/25
20244/20244 [==============================] - 28s - loss: 0.6480 - acc: 0.7212 - val_loss: 0.9547 - val_acc: 0.6862
Epoch 2/25
20244/20244 [==============================] - 28s - loss: 0.6399 - acc: 0.7229 - val_loss: 0.9653 - val_acc: 0.6743
Epoch 3/25
20244/20244 [==============================] - 28s - loss: 0.6334 - acc: 0.7292 - val_loss: 0.9605 - val_acc: 0.6798
Epoch 4/25
20244/20244 [==============================] - 29s - loss: 0.6470 - acc: 0.7172 - val_loss: 0.9661 - val_acc: 0.6706
Epoch 5/25
20244/20244 [==============================] - 29s - loss: 0.6344 - acc: 0.7266 - val_loss: 0.9708 - val_acc: 0.6734
Epoch 6/25
20244/20244 [==============================] - 29s - loss: 0.6353 - acc: 0.7250 - val_loss: 0.9766 - val_acc: 0.6683
Epoch 7/25
20244/20244 [==============================] - 29s - loss: 0.6305 - acc: 0.7286 - val_loss: 0.9709 - val_acc: 0.6706
Epoch 8/25
20244/20244 [==============================]

In [100]:
denseModel.save_weights('Dense_model_with_dropout_val_66.h5',)

In [43]:
def genDenseModelWithoutDropOut(layer):
    baseModel = Sequential()
    baseModel.add(MaxPooling2D(input_shape=layer.output_shape[1:]))
    #baseModel.add(Convolution2D(32, 3, 3, activation='relu'))
    #baseModel.add(BatchNormalization(axis=1))
    #baseModel.add(MaxPooling2D(pool_size=(2,2)))
    #baseModel.add(Convolution2D(64, 3, 3, activation='relu'))
    #baseModel.add(BatchNormalization(axis=1))
    #baseModel.add(MaxPooling2D(pool_size=(2,2)))
    baseModel.add(Flatten())
    baseModel.add(Dropout(0.))
    baseModel.add(Dense(256,activation='relu'))
    baseModel.add(BatchNormalization())
    baseModel.add(Dropout(0.))
    baseModel.add(Dense(256,activation='relu'))
    baseModel.add(BatchNormalization())
    baseModel.add(Dropout(0.))
    baseModel.add(Dense(10,activation='softmax'))
    
    baseModel.compile(Adam(lr=0.0001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return baseModel

In [46]:
denseModelWithoutDropout = genDenseModel(vggConvLayersModel.layers[-1])

In [103]:
denseModelWithoutDropout.compile(Adam(lr=0.001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModelWithoutDropout.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=3,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/3
20244/20244 [==============================] - 28s - loss: 2.8855 - acc: 0.2566 - val_loss: 1.1289 - val_acc: 0.5651
Epoch 2/3
20244/20244 [==============================] - 28s - loss: 1.4187 - acc: 0.4661 - val_loss: 0.9799 - val_acc: 0.6431
Epoch 3/3
20244/20244 [==============================] - 29s - loss: 1.1284 - acc: 0.5434 - val_loss: 0.9304 - val_acc: 0.7330


In [104]:
denseModelWithoutDropout.compile(Adam(lr=0.001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModelWithoutDropout.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=10,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/10
20244/20244 [==============================] - 28s - loss: 1.0862 - acc: 0.5516 - val_loss: 1.0369 - val_acc: 0.6179
Epoch 2/10
20244/20244 [==============================] - 28s - loss: 0.9827 - acc: 0.5822 - val_loss: 0.9942 - val_acc: 0.6716
Epoch 3/10
20244/20244 [==============================] - 29s - loss: 0.9115 - acc: 0.6112 - val_loss: 1.0208 - val_acc: 0.6596
Epoch 4/10
20244/20244 [==============================] - 28s - loss: 0.8645 - acc: 0.6288 - val_loss: 0.8915 - val_acc: 0.7174
Epoch 5/10
 1216/20244 [>.............................] - ETA: 24s - loss: 0.7752 - acc: 0.6628

KeyboardInterrupt: 

In [105]:
denseModelWithoutDropout.compile(Adam(lr=0.0001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModelWithoutDropout.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=5,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/5
20244/20244 [==============================] - 28s - loss: 0.7939 - acc: 0.6577 - val_loss: 0.9803 - val_acc: 0.6922
Epoch 2/5
20244/20244 [==============================] - 28s - loss: 0.7617 - acc: 0.6671 - val_loss: 0.9572 - val_acc: 0.7041
Epoch 3/5
20244/20244 [==============================] - 28s - loss: 0.7464 - acc: 0.6749 - val_loss: 0.9700 - val_acc: 0.6931
Epoch 4/5
20244/20244 [==============================] - 29s - loss: 0.7277 - acc: 0.6816 - val_loss: 0.9609 - val_acc: 0.6972
Epoch 5/5
20244/20244 [==============================] - 28s - loss: 0.7220 - acc: 0.6888 - val_loss: 0.9869 - val_acc: 0.6858


In [106]:
denseModelWithoutDropout.save_weights('Dense_model_without_dropout_val_69.h5')

In [41]:
def genDenseModelWithDrop(layer):
    baseModel = Sequential()
    baseModel.add(MaxPooling2D(input_shape=layer.output_shape[1:]))
    #baseModel.add(Convolution2D(32, 3, 3, activation='relu'))
    #baseModel.add(BatchNormalization(axis=1))
    #baseModel.add(MaxPooling2D(pool_size=(2,2)))
    #baseModel.add(Convolution2D(64, 3, 3, activation='relu'))
    #baseModel.add(BatchNormalization(axis=1))
    #baseModel.add(MaxPooling2D(pool_size=(2,2)))
    baseModel.add(Flatten())
    baseModel.add(Dropout(0.2))
    baseModel.add(Dense(256,activation='relu'))
    baseModel.add(BatchNormalization())
    baseModel.add(Dropout(0.4))
    baseModel.add(Dense(256,activation='relu'))
    baseModel.add(BatchNormalization())
    baseModel.add(Dropout(0.8))
    baseModel.add(Dense(10,activation='softmax'))
    
    baseModel.compile(Adam(lr=0.0001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return baseModel

In [37]:
denseModelwithDrop = genDenseModelWithDrop(vggConvLayersModel.layers[-1])

In [33]:
denseModelwithDrop.compile(Adam(lr=0.001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModelwithDrop.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=5,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/5
20244/20244 [==============================] - 44s - loss: 1.6081 - acc: 0.4380 - val_loss: 1.4190 - val_acc: 0.4560
Epoch 2/5
20244/20244 [==============================] - 23s - loss: 0.9294 - acc: 0.6062 - val_loss: 0.9593 - val_acc: 0.6257
Epoch 3/5
20244/20244 [==============================] - 24s - loss: 0.7634 - acc: 0.6615 - val_loss: 0.9554 - val_acc: 0.6752
Epoch 4/5
20244/20244 [==============================] - 24s - loss: 0.6523 - acc: 0.7111 - val_loss: 1.0930 - val_acc: 0.5518
Epoch 5/5
20244/20244 [==============================] - 24s - loss: 0.5805 - acc: 0.7487 - val_loss: 1.0472 - val_acc: 0.5647


In [34]:
denseModelwithDrop.compile(Adam(lr=0.00001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModelwithDrop.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=5,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/5
20244/20244 [==============================] - 23s - loss: 0.4715 - acc: 0.7987 - val_loss: 1.0130 - val_acc: 0.6161
Epoch 2/5
20244/20244 [==============================] - 23s - loss: 0.4441 - acc: 0.8105 - val_loss: 0.9971 - val_acc: 0.6243
Epoch 3/5
20244/20244 [==============================] - 23s - loss: 0.4293 - acc: 0.8251 - val_loss: 0.9805 - val_acc: 0.6367
Epoch 4/5
20244/20244 [==============================] - 23s - loss: 0.4168 - acc: 0.8291 - val_loss: 0.9911 - val_acc: 0.6399
Epoch 5/5
20244/20244 [==============================] - 23s - loss: 0.4080 - acc: 0.8359 - val_loss: 0.9904 - val_acc: 0.6367


In [38]:
denseModelwithDrop.compile(Adam(lr=0.0001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModelwithDrop.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=10,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/10
20244/20244 [==============================] - 23s - loss: 2.3514 - acc: 0.2896 - val_loss: 1.0994 - val_acc: 0.6193
Epoch 2/10
20244/20244 [==============================] - 23s - loss: 1.5046 - acc: 0.4801 - val_loss: 1.1097 - val_acc: 0.6050
Epoch 3/10
20244/20244 [==============================] - 24s - loss: 1.2177 - acc: 0.5515 - val_loss: 1.0764 - val_acc: 0.6202
Epoch 4/10
20244/20244 [==============================] - 24s - loss: 1.0583 - acc: 0.5923 - val_loss: 1.0168 - val_acc: 0.6468
Epoch 5/10
20244/20244 [==============================] - 24s - loss: 0.9595 - acc: 0.6209 - val_loss: 0.9974 - val_acc: 0.6408
Epoch 6/10
20244/20244 [==============================] - 24s - loss: 0.8854 - acc: 0.6453 - val_loss: 0.9042 - val_acc: 0.6784
Epoch 7/10
20244/20244 [==============================] - 24s - loss: 0.8110 - acc: 0.6697 - val_loss: 0.9513 - val_acc: 0.6431
Epoch 8/10
20244/20244 [==============================]

In [42]:
denseModelwithDrop.compile(Adam(lr=0.0001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
denseModelwithDrop.fit(NepredictTrainVggConv,train_label_arr,batch_size=64,nb_epoch=10,verbose=1,validation_data=(predictValidationVggConv,valid_label_arr))

Train on 20244 samples, validate on 2180 samples
Epoch 1/10
20244/20244 [==============================] - 24s - loss: 0.4475 - acc: 0.8196 - val_loss: 1.2597 - val_acc: 0.5445
Epoch 2/10
20244/20244 [==============================] - 24s - loss: 0.4166 - acc: 0.8300 - val_loss: 1.1192 - val_acc: 0.5431
Epoch 3/10
20244/20244 [==============================] - 24s - loss: 0.4010 - acc: 0.8402 - val_loss: 1.1579 - val_acc: 0.5904
Epoch 4/10
20244/20244 [==============================] - 24s - loss: 0.3794 - acc: 0.8473 - val_loss: 1.1888 - val_acc: 0.5564
Epoch 5/10
10112/20244 [=============>................] - ETA: 11s - loss: 0.3594 - acc: 0.8567

KeyboardInterrupt: 

# Predictions

In [47]:
denseModelWithoutDropout.load_weights('Dense_model_without_dropout_val_69.h5')
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)


In [48]:
def do_clip(arr,mx): return np.clip(arr,(1-mx)/9, mx)


In [52]:
predictTest = denseModelWithoutDropout.predict(predict_test_on_VggConv)

In [53]:
predictTest[:5]

array([[  1.5315e-03,   4.1643e-04,   1.4970e-04,   1.1657e-04,   4.3386e-04,   4.0016e-03,
          2.7593e-02,   1.5262e-01,   6.7707e-01,   1.3607e-01],
       [  1.0585e-01,   1.9827e-03,   2.0686e-04,   8.2657e-04,   6.4406e-03,   2.2795e-02,
          8.9233e-03,   1.2223e-02,   2.1527e-01,   6.2548e-01],
       [  2.8955e-02,   4.7932e-03,   5.4716e-02,   4.4092e-01,   3.7671e-01,   1.2082e-02,
          2.7328e-03,   6.6634e-03,   2.8934e-02,   4.3491e-02],
       [  1.7397e-03,   4.2695e-03,   2.4025e-03,   1.6384e-03,   4.2070e-03,   6.2756e-02,
          5.6232e-01,   2.9445e-01,   6.0709e-02,   5.5036e-03],
       [  1.8244e-02,   2.3785e-03,   6.5692e-04,   6.7714e-04,   2.0919e-03,   3.2355e-03,
          2.7286e-03,   4.1974e-02,   6.7876e-01,   2.4925e-01]], dtype=float32)

In [54]:
clip_test_predict = do_clip(predictTest, 0.93)

In [56]:
classes = sorted(val_batches.class_indices, key=val_batches.class_indices.get)
classes

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [100]:
%cd ..

/home/ubuntu/courses/deeplearning1/nbs


In [99]:
subm_name006 = 'kag6.csv'

In [58]:
submission = pd.DataFrame(clip_test_predict, columns=classes)
submission.head()

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.027593,0.152616,0.677074,0.136067
1,0.105853,0.007778,0.007778,0.007778,0.007778,0.022795,0.008923,0.012223,0.215273,0.625476
2,0.028955,0.007778,0.054716,0.440919,0.376712,0.012082,0.007778,0.007778,0.028934,0.043491
3,0.007778,0.007778,0.007778,0.007778,0.007778,0.062756,0.562319,0.294455,0.060709,0.007778
4,0.018244,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.041974,0.678765,0.249249


In [60]:
test_file_names_arr=test_batch.filenames

In [89]:
submission.to_csv(subm_name006, index=False)

In [102]:
FileLink('data/statefarm/'+subm_name006)

/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/kag6.csv

# With More Data

In [103]:
imgGen = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                                  shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

In [105]:

augmentTrainBatch = imgGen.flow_from_directory(directory=path+'train', batch_size=32, target_size=(224,224), 
                                               shuffle=False, class_mode='categorical')

Found 20244 images belonging to 10 classes.


In [106]:
# predict X5 the augmented train batch on VGG16 conv layers
predict_5X_augment_train_vgg_conv = vggConvLayersModel.predict_generator(augmentTrainBatch, augmentTrainBatch.N*3)

In [107]:
train_labels_arr = onehot(augmentTrainBatch.classes)

In [108]:
X5_train_labels_arr = np.concatenate([train_labels_arr]*3)

In [109]:
save_array('X5_train_labels_arr.dat', X5_train_labels_arr)

In [110]:
save_array('predict_5X_augment_train_vgg_conv.dat', predict_5X_augment_train_vgg_conv)

In [111]:
predict_5X_augment_train_vgg_conv = load_carray('predict_5X_augment_train_vgg_conv.dat')

In [112]:
X5_train_labels_arr = load_array('X5_train_labels_arr.dat')

In [113]:
predict_5X_augment_train_vgg_conv.shape

(60732, 512, 14, 14)

In [114]:
X5_train_labels_arr.shape

(60732, 10)

In [115]:
denseModelWithoutDropout.fit(predict_5X_augment_train_vgg_conv[:], X5_train_labels_arr,  nb_epoch=1, verbose=1, 
               validation_data=(predictValidationVggConv,valid_label_arr))

Train on 60732 samples, validate on 2180 samples
Epoch 1/1
60732/60732 [==============================] - 40s - loss: 0.3663 - acc: 0.8910 - val_loss: 0.5005 - val_acc: 0.8206


In [116]:
denseModelWithoutDropout.optimizer.lr=0.000001

In [117]:
denseModelWithoutDropout.fit(predict_5X_augment_train_vgg_conv[:], X5_train_labels_arr,  nb_epoch=25, verbose=1, 
               validation_data=(predictValidationVggConv,valid_label_arr))

Train on 60732 samples, validate on 2180 samples
Epoch 1/25
60732/60732 [==============================] - 44s - loss: 0.2398 - acc: 0.9280 - val_loss: 0.5103 - val_acc: 0.8367
Epoch 2/25
60732/60732 [==============================] - 42s - loss: 0.1872 - acc: 0.9434 - val_loss: 0.4909 - val_acc: 0.8413
Epoch 3/25
60732/60732 [==============================] - 40s - loss: 0.1608 - acc: 0.9512 - val_loss: 0.4363 - val_acc: 0.8541
Epoch 4/25
60732/60732 [==============================] - 39s - loss: 0.1342 - acc: 0.9597 - val_loss: 0.4480 - val_acc: 0.8514
Epoch 5/25
60732/60732 [==============================] - 39s - loss: 0.1202 - acc: 0.9644 - val_loss: 0.4577 - val_acc: 0.8587
Epoch 6/25
60732/60732 [==============================] - 38s - loss: 0.1052 - acc: 0.9682 - val_loss: 0.4937 - val_acc: 0.8546
Epoch 7/25
60732/60732 [==============================] - 38s - loss: 0.0978 - acc: 0.9701 - val_loss: 0.4607 - val_acc: 0.8573
Epoch 8/25
60732/60732 [==============================]

In [118]:
denseModelWithoutDropout.save_weights('State_Farm_Trained_val_acc_0.8867_over_fit.h5')

In [120]:
denseModelWithoutDropout.load_weights('State_Farm_Trained_val_acc_0.8867_over_fit.h5')

In [121]:
def do_clip(arr, mx): return np.clip(arr, ((1-mx)/9.0), mx)

In [122]:
#Making predictions
predictTest = denseModelWithoutDropout.predict(predict_test_on_VggConv)

In [123]:
clip_test_predict = do_clip(predictTest, 0.93)

In [124]:
clip_test_predict[:2]

array([[ 0.0078,  0.0078,  0.0078,  0.0078,  0.0078,  0.0078,  0.0195,  0.03  ,  0.1232,  0.8266],
       [ 0.0078,  0.0078,  0.0078,  0.0078,  0.0078,  0.0078,  0.0078,  0.0078,  0.0078,  0.93  ]], dtype=float32)

In [125]:
classes = sorted(val_batches.class_indices, key=val_batches.class_indices.get)

In [142]:
subm_with_more_data02 = 'state_farm_more_data.csv'

In [127]:
submission = pd.DataFrame(clip_test_predict, columns=classes)
submission.head()

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.019516,0.030042,0.123151,0.826587
1,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.930000
2,0.007778,0.007778,0.007778,0.007778,0.930000,0.007778,0.007778,0.007778,0.007778,0.007778
3,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.193114,0.007778,0.804440,0.007778
4,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.582465,0.408116


In [141]:
submission.insert(0, 'img', [a[8:] for a in test_file_names_arr])
submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.019516,0.030042,0.123151,0.826587
1,img_14887.jpg,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.930000
2,img_62885.jpg,0.007778,0.007778,0.007778,0.007778,0.930000,0.007778,0.007778,0.007778,0.007778,0.007778
3,img_45125.jpg,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.193114,0.007778,0.804440,0.007778
4,img_22633.jpg,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.582465,0.408116


In [143]:
submission.to_csv(subm_with_more_data02, index=False)

In [137]:
%pwd

u'/home/ubuntu/courses/deeplearning1/nbs'

In [138]:
FileLink('/data/statefarm/'+subm_with_more_data)

/data/statefarm/More_data_data.csv

# Pseudo Labeling

In [10]:
pseudo_val = load_array("predictValidationVggConv")

In [11]:
predict_5X_augment_train_vgg_conv = load_carray('predict_5X_augment_train_vgg_conv.dat')

In [14]:
train_w_pasudo_input = predict_test_on_VggConv.copy()

MemoryError: 